<a href="https://colab.research.google.com/github/LucasColas/Project-with-CIFAR-10/blob/main/Training_Project_CIFAR_10_Data_Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Les modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.applications.vgg16 import VGG16

VGG_model = VGG16(weights="imagenet", include_top=False, input_shape=(32,32,3))
#VGG_model.summary()

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print(X_train.dtype, y_train.shape, X_test.shape, y_test.shape)

170500096/170498071 [==============================] - 4s 0us/step
float32 (50000, 1) (10000, 32, 32, 3) (10000, 1)


Data Augmentation

In [8]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,rotation_range=20,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         horizontal_flip=True,
                         validation_split=0.2)
aug.fit(X_train)

One Hot Encoding

In [3]:
def sort_labels(y_train, y_test):

    Y_train = np.zeros((50000,10))
    Y_test = np.zeros((10000,10))
    for index, label in enumerate(y_train):
        Y_train[index, label] = 1

    for index, label in enumerate(y_test):
        Y_test[index, label] = 1


    return Y_train, Y_test

Y_train, Y_test = sort_labels(y_train, y_test)

Unfreeze les derniers layers

In [4]:
VGG_model.trainable = True
set_trainable = False
for layer in VGG_model.layers:
  #print(layer.name)
  if layer.name == 'block5_conv3':
    set_trainable = True
  
  if set_trainable:
    layer.trainable = True 

  else:
    layer.trainable = False

Preprocessing

In [5]:
X_train /= 255
X_test /= 255

Entrainemenet avec LR élevé

In [12]:
model = models.Sequential()
model.add(VGG_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

lr = 0.01
steps_per_epoch = np.ceil((0.8*len(X_train) / 16)-1)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr), metrics=["acc"])
model.fit(aug.flow(X_train, Y_train, batch_size=16,subset='training'), validation_data=aug.flow(X_train, Y_train,
         batch_size=8, subset='validation'), steps_per_epoch=steps_per_epoch, epochs=10, verbose=1)

Epoch 1/10
2499/2499 [==============================] - 41s 16ms/step - loss: 1.1692 - acc: 0.5918 - val_loss: 1.0867 - val_acc: 0.6129
Epoch 2/10
2499/2499 [==============================] - 40s 16ms/step - loss: 1.0681 - acc: 0.6254 - val_loss: 1.0042 - val_acc: 0.6486
Epoch 3/10
2499/2499 [==============================] - 41s 16ms/step - loss: 1.0344 - acc: 0.6424 - val_loss: 1.0534 - val_acc: 0.6285
Epoch 4/10
2499/2499 [==============================] - 41s 16ms/step - loss: 1.0134 - acc: 0.6462 - val_loss: 0.9977 - val_acc: 0.6495
Epoch 5/10
2499/2499 [==============================] - 41s 16ms/step - loss: 0.9937 - acc: 0.6526 - val_loss: 0.9864 - val_acc: 0.6514
Epoch 6/10
2499/2499 [==============================] - 41s 16ms/step - loss: 0.9764 - acc: 0.6587 - val_loss: 0.9984 - val_acc: 0.6535
Epoch 7/10
2499/2499 [==============================] - 41s 16ms/step - loss: 0.9611 - acc: 0.6609 - val_loss: 0.9462 - val_acc: 0.6685
Epoch 8/10
2499/2499 [==========================

Entrainement de toutes les couches

In [13]:
for layer in model.layers:
  layer.trainable = True



In [14]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                2570      
Total params: 14,848,586
Trainable params: 14,848,586
Non-trainable params: 0
_________________________________________________________________


In [15]:

model = models.Sequential()
model.add(VGG_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 14,848,586
Trainable params: 14,848,586
Non-trainable params: 0
_________________________________________________________________


In [ ]:


steps_per_epoch = np.ceil((0.8*len(X_train) / 16)-1)
model.compile(optimizer=optimizers.SGD(lr=0.0001),loss="categorical_crossentropy", metrics=["acc"])
history = model.fit(aug.flow(X_train, Y_train, batch_size=16,subset='training'), validation_data=aug.flow(X_train, Y_train,
         batch_size=8, subset='validation'), steps_per_epoch=steps_per_epoch, epochs=10, verbose=1)
model.save("training.h5")



acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs,acc,'bo', label='Training acc')
plt.plot(epochs, val_acc,'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/10
2499/2499 [==============================] - 72s 28ms/step - loss: 1.8411 - acc: 0.3432 - val_loss: 1.3219 - val_acc: 0.5396
Epoch 2/10
2499/2499 [==============================] - 72s 29ms/step - loss: 1.2958 - acc: 0.5485 - val_loss: 1.0644 - val_acc: 0.6290
Epoch 3/10
2499/2499 [==============================] - 72s 29ms/step - loss: 1.1127 - acc: 0.6150 - val_loss: 0.9607 - val_acc: 0.6721
Epoch 4/10
2499/2499 [==============================] - 72s 29ms/step - loss: 1.0193 - acc: 0.6488 - val_loss: 0.9033 - val_acc: 0.6818
Epoch 5/10
2499/2499 [==============================] - 73s 29ms/step - loss: 0.9589 - acc: 0.6719 - val_loss: 0.8650 - val_acc: 0.7008
Epoch 6/10
2499/2499 [==============================] - 73s 29ms/step - loss: 0.9171 - acc: 0.6857 - val_loss: 0.8457 - val_acc: 0.7057
Epoch 7/10
2499/2499 [==============================] - 72s 29ms/step - loss: 0.8918 - acc: 0.6949 - val_loss: 0.8199 - val_acc: 0.7155
Epoch 8/10
2499/2499 [==========================